# Coroutines

Generators produce values - Coroutines consume values (may not return anything)

Coroutines are not for iterating over sequences. 
A coroutine is built from a generator (technically a coroutine object is a generator object), but it is conceptually different:
a coroutine is designed to repeatedly send input to it, process that input and stop at yield statement.

Coroutine also may sound similar to function, but there is important difference:
- function: it's the same function each time it is called
- coroutine: persistent properties can be referenced and altered
Maintaining and using the state is what a coroutine object does. Similar to function it can be called and produce the value, but its most important ability is to change the state of its own properties, the state of something else or both.

## send() method

send() is a method that was added to generators exclusively for the purpose of coroutine functionality.
In coroutines the yield statement is used to capture the value of whatever is passed to the send() method.
In the case of coroutines the yield statement is not only in charge of pausing the flow, but also capturing values.
(We use yield to generate values in generators and to consume values in coroutines).

In [1]:
def finder(x):
  while True:
    input_text = yield
    if input_text in x:
        print(f'{input_text} found in {x}')
    else:
        print(f'{input_text} not found in {x}')

In [2]:
f = finder('California')
type(f)           # generator object

generator

In [3]:
f.send('Cali')

TypeError: can't send non-None value to a just-started generator

### Priming the coroutine

Before start using coroutine we should initialize it (or prime).
We can prime it in 2 ways: `next(f)` or `f.send(None)`

In [4]:
next(f)

In [5]:
f.send('Cali')

Cali found in California


In [6]:
f.send('asdf')

asdf not found in California


In [7]:
f.close() # shuts down the coroutine

In [8]:
# also we can throw exception
f.throw(AttributeError, "Doesn't have to be an AttributeError, can be anything")

AttributeError: Doesn't have to be an AttributeError, can be anything

### Example 1

In [9]:
def counter(string):
    count = 0
    try:
        while True:
            item = yield
            if isinstance(item, str):
                if item in string:
                    count += 1
                    print(item)
                else:
                    print('No match')
            else:
                print('Not a string')
    except GeneratorExit:
        print(count)
        
c = counter('California')
next(c)
c.send('Cali')  # Cali
c.send('nia')   # nia
c.send('Hawaii')# No match
c.send(1234)    # Not a string
c.close()       # 2

Cali
nia
No match
Not a string
2


## Priming coroutine using decorator

In [10]:
def coroutine_decorator(func):
    def wrapper(*args, **kwargs):
        cr = func(*args, **kwargs)
        next(cr)
        return cr
    return wrapper

In [11]:
@coroutine_decorator
def coroutine_example():
    while True:
        x = yield
        # do something with x
        print(x)

In [12]:
c = coroutine_example()
c.send('asdf')

asdf


## Consume values with send() method

In [13]:
def sender(filename, cr):
    for line in open(filename):
        cr.send(line)
    cr.close()

In [14]:
@coroutine_decorator
def match_counter(string):
    count = 0
    try:
        while True:
            line = yield
            if string in line:
                count += 1
    except GeneratorExit:
        print(f'{string}: {count}')

In [15]:
c = match_counter('Da')
sender('coroutines_files/names.txt', c)

Da: 5


In [16]:
@coroutine_decorator
def longer_than(n):
    count = 0
    try:
        while True:
            line = yield
            if len(line) > n:
                print(line)
                count += 1
    except GeneratorExit:
        print(f'{count} lines longer than {n} characters')

In [17]:
l = longer_than(18)
sender('coroutines_files/names.txt', l)

joLahoma Mondragon

Tanesha Finkbeiner

Brittanie Talamantes

3 lines longer than 18 characters


## Coroutine pipelines

In [18]:
@coroutine_decorator
def router():
    try:
        while True:
            line = yield
            first, last = line.split(' ')
            fnames.send(first)
            lnames.send(last.strip())
    except GeneratorExit:
        fnames.close()
        lnames.close()

In [19]:
@coroutine_decorator
def file_write(filename):
    try:
        with open(filename, 'a') as f:
            while True:
                line = yield
                f.write(line+'\n')
    except GeneratorExit:
        f.close()
        print('file created')

In [20]:
fnames = file_write('coroutines_files/first_names.txt')
lnames = file_write('coroutines_files/last_names.txt')
router = router()
for line in open('coroutines_files/names.txt'):
    router.send(line)
router.close()

file created
file created
